In this basic scenario, we will be simulating a small data center equipped with
photovoltaic (PV) modules and a simple battery system. First, we need to import
all necessary vessim modules and datasets, we will be using for this scenario.

In [1]:
from _data import load_carbon_data, load_solar_data
from vessim.actor import ComputingSystem, Generator
from vessim.controller import Monitor
from vessim.cosim import Environment, Microgrid
from vessim.power_meter import MockPowerMeter
from vessim.signal import HistoricalSignal
from vessim.storage import SimpleBattery

# Jupyter async bug fix
import nest_asyncio
nest_asyncio.apply()

The vessim Environment holds all simulation components and is defined at the
very start of our scenario. We initialize it with the start date of our
simulation. The battery of our small datacenter is also rather small and even
with our PV modules will likely not suffice to indefinetly suppply all required
power. Our datacenter will, at some point, have to fall back to consuming power
from the eletrical grid, which comes with its own carbon footprint. We simulate
this behaviour by adding a grid signal with historical carbon data to our
Environment.

In [2]:
environment = Environment(sim_start="2020-06-11 00:00:00")
environment.add_grid_signal("carbon_intensity", HistoricalSignal(load_carbon_data()))

The battery of our small datacenter is also rather small and even with our PV
modules will likely not suffice to indefinetly suppply all required power. Our
datacenter will, at some point, have to fall back to consuming power from the
eletrical grid, which comes with its own carbon footprint. We simulate this
behaviour by adding a grid signal with historical carbon data to our
Environment.

In [3]:
environment.add_grid_signal("carbon_intensity", HistoricalSignal(load_carbon_data()))

Central to our vessim Environment and simulation is the Microgrid component. The
Microgrid holds and connects all Actors, Controllers and Storage entities.
Actors can consume and produce power. This scenario includes two Actors: a
computing system with two power consuming nodes, and a generator which produces
power based on historic solar data, turning it into a PV module. An excess or
deficiency in power can be fed to or drawn from Storage, which in this case is a
simple battery. Controller entities can react to the Actors activity and the
total power delta that is drawn from or fed to the electrical grid. For now, the
only controller is a monitor, which stores all simulation data in a CSV file.


In [3]:
monitor = Monitor()  # stores simulation result on each step
microgrid = Microgrid(
    actors=[
        ComputingSystem(power_meters=[
            MockPowerMeter(p=3),
            MockPowerMeter(p=7)
        ]),
        Generator(signal=HistoricalSignal(load_solar_data(sqm=0.4 * 0.5))),
    ],
    controllers=[monitor],
    storage=SimpleBattery(capacity=100),
    zone="DE",
    step_size=60,  # global step size (can be overridden by actors or controllers)
)
environment.add_microgrid(microgrid)

environment.run(until=3600 * 24 * 2) # two days
monitor.to_csv("result.csv")

2024-02-05 12:59:44.217 | INFO     | mosaik.scenario:start:280 - Starting "Grid" as "Grid-0" ...
2024-02-05 12:59:44.218 | INFO     | mosaik.scenario:start:280 - Starting "Actor" as "Actor-0" ...
2024-02-05 12:59:44.219 | INFO     | mosaik.scenario:start:280 - Starting "Actor" as "Actor-1" ...
2024-02-05 12:59:44.219 | INFO     | mosaik.scenario:start:280 - Starting "Controller" as "Controller-0" ...
2024-02-05 12:59:44.221 | INFO     | mosaik.scenario:run:598 - Starting simulation.
100%|██████████| 172800/172800 [00:01<00:00, 165125.56steps/s]
2024-02-05 12:59:45.270 | INFO     | mosaik.scenario:run:646 - Simulation finished successfully.


TODO show nice plot and interpret a little